In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#Daadwerkelijke comment in "body"

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install vaderSentiment
!pip install ndjson

In [5]:
import os
import ndjson
import re
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

# Define the dataset path and the ndjson file path
dataset_path = 'data/comments.ndjson'
ndjson_file_path = os.path.join(dataset_path, 'artificialinteligence_comments.ndjson')

# Load the NDJSON file containing data
with open(dataset_path, 'r') as f:
    json_data = ndjson.load(f)  # Parses the entire NDJSON file into a Python list

# Example: Extract 'body' field if present
bodies = [entry['body'] for entry in json_data if 'body' in entry]

# Print the first few bodies
print(bodies[:5])  # Adjust the slicing as needed

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 108342: character maps to <undefined>

Hier moet nog code komen om alles met regulation eruit te krijgen, Lijst maken met hoe en wat. 

In [6]:
output_dir = 'output'  # Directory to store output files
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

output_file_path = os.path.join(output_dir, 'matched_entries.ndjson')
csv_file_path = os.path.join(output_dir, 'matched_entries.csv')
first_100_ndjson_path = os.path.join(output_dir, 'first_100_matched_entries.ndjson')

# Define the regex pattern using the initial set of keywords
pattern = (
    r"\b(?:"
    r"Regulat(?:ion|ions|ory)|"    # matches "Regulation", "Regulations", "Regulatory"
    r"Law(?:s)?|"                  # matches "Law" or "Laws"
    r"Government|"
    r"Polic(?:y|ies)"              # matches "Policy" or "Policies"
    r")\b"
)

# Compile the regex pattern with case-insensitive flag
try:
    regex = re.compile(pattern, re.IGNORECASE)
except re.error as e:
    print(f"Regex compilation error: {e}")
    exit(1)

# Load the NDJSON data with error handling
try:
    with open(ndjson_file_path, 'r', encoding='utf-8') as f:
        json_data = ndjson.load(f)  # Parses the entire NDJSON file into a Python list of dictionaries
except FileNotFoundError:
    print(f"Error: The file {ndjson_file_path} does not exist.")
    exit(1)
except ndjson.JSONDecodeError:
    print(f"Error: The file {ndjson_file_path} is not a valid NDJSON file.")
    exit(1)

# Filter the entries that match at least one keyword in their body
matched_entries = [
    entry for entry in json_data 
    if 'body' in entry and regex.search(entry['body'])
]

# Write matched entries to an NDJSON file
try:
    with open(output_file_path, 'w', encoding='utf-8') as f:
        ndjson.dump(matched_entries, f)
except Exception as e:
    print(f"Error writing to {output_file_path}: {e}")
    exit(1)

# Create a DataFrame from the matched entries
df = pd.DataFrame(matched_entries)

# Save the matched entries to a CSV file
try:
    df.to_csv(csv_file_path, index=False, encoding='utf-8')
except Exception as e:
    print(f"Error writing to {csv_file_path}: {e}")
    exit(1)

# Save the first 100 matched entries to another NDJSON file
first_100_entries = matched_entries[:100]
try:
    with open(first_100_ndjson_path, 'w', encoding='utf-8') as f:
        ndjson.dump(first_100_entries, f)
except Exception as e:
    print(f"Error writing to {first_100_ndjson_path}: {e}")
    exit(1)

# Print summary
print(f"Found {len(matched_entries)} bodies containing at least one regulatory/policy keyword.")
print("Sample matched bodies:", [e['body'] for e in matched_entries[:3]])
print("Matched entries saved to:")
print(f"  NDJSON: {output_file_path}")
print(f"  CSV: {csv_file_path}")
print(f"  First 100 NDJSON: {first_100_ndjson_path}")


Error: The file data/comments.ndjson\artificialinteligence_comments.ndjson does not exist.


NameError: name 'json_data' is not defined

# VaderSentiment

In [5]:
# Initialize the Sentiment Intensity Analyzer
analyzer = SentimentIntensityAnalyzer()

# Convert to a DataFrame
df = pd.DataFrame(bodies, columns=['body'])

matched_entries = []
for entry in json_data:
    if 'body' in entry and regex.search(entry['body']):
        # Safely get 'id' if it exists
        _id = entry['id'] if 'id' in entry else None
        matched_entries.append((_id, entry['body']))

# -------------------------------------------------------------------
# 4) Create a DataFrame with 'id' and 'body' columns
# -------------------------------------------------------------------
df_matched = pd.DataFrame(matched_entries, columns=['id', 'body'])

In [6]:
# Initialize counters for each sentiment
positive_count = 0
negative_count = 0
neutral_count = 0

# Analyze each text and classify
for text in df_matched['body']:
    scores = analyzer.polarity_scores(text)
    
    # Classify based on compound score
    if scores['compound'] > 0.05:
        sentiment = "Positive"
        positive_count += 1
    elif scores['compound'] < -0.05:
        sentiment = "Negative"
        negative_count += 1
    else:
        sentiment = "Neutral"
        neutral_count += 1

# Output the counts of each sentiment
print(f"Positive Comments: {positive_count}")
print(f"Negative Comments: {negative_count}")
print(f"Neutral Comments: {neutral_count}")

Positive Comments: 177
Negative Comments: 80
Neutral Comments: 37


Naive Bayes

In [14]:
# Improved Preprocessing Function
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z]", " ", text)
    # Convert to lowercase and tokenize
    tokens = text.lower().split()
    return ' '.join(tokens)

# Load the datasets from GitHub
train_url = "https://raw.githubusercontent.com/tyqiangz/multilingual-sentiment-datasets/main/data/english/train.tsv"
valid_url = "https://raw.githubusercontent.com/tyqiangz/multilingual-sentiment-datasets/main/data/english/valid.tsv"
test_url = "https://raw.githubusercontent.com/tyqiangz/multilingual-sentiment-datasets/main/data/english/test.tsv"

df_train = pd.read_csv(train_url, sep="\t", header=0)
df_valid = pd.read_csv(valid_url, sep="\t", header=0)
df_test = pd.read_csv(test_url, sep="\t", header=0)

# Check and remove rows with NaN values for all datasets
df_train.dropna(subset=["text", "label"], inplace=True)
df_valid.dropna(subset=["text", "label"], inplace=True)
df_test.dropna(subset=["text", "label"], inplace=True)

# Ensure that labels are strings for consistency
df_train["label"] = df_train["label"].astype(str)
df_valid["label"] = df_valid["label"].astype(str)
df_test["label"] = df_test["label"].astype(str)

# Preprocess the text data
df_train["text"] = df_train["text"].apply(preprocess_text)
df_valid["text"] = df_valid["text"].apply(preprocess_text)
df_test["text"] = df_test["text"].apply(preprocess_text)

# Combine train and valid for grid search
X_combined = pd.concat([df_train["text"], df_valid["text"]])
y_combined = pd.concat([df_train["label"], df_valid["label"]])

# Define parameter grid for TfidfVectorizer and MultinomialNB
param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'vectorizer__max_features': [3000, 5000, 7000],
    'classifier__alpha': [0.1, 0.5, 1.0]
}

# Create a pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),  # Bag-of-Words extractor
    ('classifier', MultinomialNB())
])

# Grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_combined, y_combined)

# Best model
best_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

# Evaluate on the test set
X_test_tfidf = best_model.named_steps['vectorizer'].transform(df_test["text"])
y_test_pred = best_model.named_steps['classifier'].predict(X_test_tfidf)
print("Test Classification Report:")
print(classification_report(df_test["label"], y_test_pred, zero_division=0))

# Provide a summary of test predictions
positive_count = (y_test_pred == "positive").sum()
negative_count = (y_test_pred == "negative").sum()
neutral_count = (y_test_pred == "neutral").sum()

print(f"\nSummary of Test Predictions:")
print(f"Positive Comments: {positive_count}")
print(f"Negative Comments: {negative_count}")
print(f"Neutral Comments: {neutral_count}")

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Parameters: {'classifier__alpha': 0.5, 'vectorizer__max_features': 5000, 'vectorizer__ngram_range': (1, 1)}
Test Classification Report:
              precision    recall  f1-score   support

    negative       0.48      0.67      0.56       290
     neutral       0.38      0.29      0.33       290
    positive       0.62      0.52      0.57       290

    accuracy                           0.49       870
   macro avg       0.49      0.49      0.48       870
weighted avg       0.49      0.49      0.48       870


Summary of Test Predictions:
Positive Comments: 243
Negative Comments: 408
Neutral Comments: 219


In [15]:
# Transform the body text using the vectorizer from the best model
X_matched_tfidf = best_model.named_steps['vectorizer'].transform(df_matched["body"])
# Predict labels using the classifier from the best model
df_matched["predicted_label"] = best_model.named_steps['classifier'].predict(X_matched_tfidf)
# print("\nPredicted Labels for df_matched:")
# print(df_matched)

# Summary of matched predictions
positive_matched = (df_matched["predicted_label"] == "positive").sum()
negative_matched = (df_matched["predicted_label"] == "negative").sum()
neutral_matched = (df_matched["predicted_label"] == "neutral").sum()

print(f"\nSummary of Matched Predictions:")
print(f"Positive Comments: {positive_matched}")
print(f"Negative Comments: {negative_matched}")
print(f"Neutral Comments: {neutral_matched}")


Summary of Matched Predictions:
Positive Comments: 15
Negative Comments: 227
Neutral Comments: 52
